In [2]:
#!g1.1
import torch
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader, random_split


In [3]:
#!g1.1
import os
import pandas as pd
import numpy as np
from PIL import Image
from tqdm.notebook import tqdm

class BhwDataset(Dataset):
    test_pr = 0.25
    def __init__(self, root, train, transform=None):
        super().__init__()
        self.transform = transform
        droot = 'trainval/trainval/'
        self.root = root + droot
        data = pd.read_csv(os.path.join(root, 'labels.csv'))
        data.sort_values(by='Id', inplace=True)
        ids = np.random.choice(data.shape[0], int(data.shape[0] * self.test_pr), replace=False)
        self.file_names = data['Id'].values
        self.labels = data['Label'].values
        if train:
            self.file_names = np.delete(self.file_names, ids)
            self.labels = np.delete(self.labels, ids)
        else:
            self.file_names = self.file_names[ids]
            self.labels = self.labels[ids]
        self.classes = np.unique(data['Label'].values)

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, id):
        item = Image.open(os.path.join(self.root, self.file_names[id])).convert('RGB')
        label = self.labels[id]
        if self.transform is not None:
            item = self.transform(item)
        return item, label
    
class BhwtestDataset(Dataset):
    def __init__(self, root, transform=None):
        super().__init__()
        self.transform = transform
        self.root = root
        self.file_names = sorted(os.listdir(root))
        self.labels = [0] * len(self.file_names)

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, id):
        item = Image.open(os.path.join(self.root, self.file_names[id])).convert('RGB')
        label = self.labels[id]
        if self.transform is not None:
            item = self.transform(item)
        return item, label


In [4]:
#!g1.1
normalize = T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

train_transform = T.Compose([
    T.RandomResizedCrop(224, scale=(0.5, 1.0)),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    normalize,
])

test_transform = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    normalize,
])

In [5]:
#!g1.1
train_dataset = BhwDataset(root='/home/jupyter/mnt/datasets/bhw1/', train=True, transform=train_transform)
test_dataset = BhwDataset(root='/home/jupyter/mnt/datasets/bhw1/', train=False, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

In [6]:
#!g1.1
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [7]:
#!g1.1

def training_epoch(model, optimizer, criterion, train_loader, tqdm_desc):
    train_loss, train_accuracy = 0.0, 0.0
    model.train()
    for images, labels in tqdm(train_loader, desc=tqdm_desc):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        logits = model(images)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.shape[0]
        train_accuracy += (logits.argmax(dim=1) == labels).sum().item()
    
    train_loss /= len(train_loader.dataset)
    train_accuracy /= len(train_loader.dataset)
    return train_loss, train_accuracy


@torch.no_grad()
def validation_epoch(model, criterion, test_loader, tqdm_desc):
    test_loss, test_accuracy = 0.0, 0.0
    model.eval()
    for images, labels in tqdm(test_loader, desc=tqdm_desc):
        images = images.to(device)
        labels = labels.to(device)
        logits = model(images)
        loss = criterion(logits, labels)

        test_loss += loss.item() * images.shape[0]
        test_accuracy += (logits.argmax(dim=1) == labels).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy /= len(test_loader.dataset)
    return test_loss, test_accuracy

@torch.no_grad()
def test_epoch(model, test_loader, tqdm_desc):
    test_loss, test_accuracy = 0.0, 0.0
    model.eval()
    for images, labels in tqdm(test_loader, desc=tqdm_desc):
        images = images.to(device)
        labels = labels.to(device)
        logits = model(images)
        loss = criterion(logits, labels)

        test_loss += loss.item() * images.shape[0]
        test_accuracy += (logits.argmax(dim=1) == labels).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy /= len(test_loader.dataset)
    return test_loss, test_accuracy

    
def train(model, optimizer, scheduler, criterion, train_loader, test_loader, num_epochs):
    train_losses, train_accuracies = [], []
    test_losses, test_accuracies = [], []

    for epoch in range(1, num_epochs + 1):
        train_loss, train_accuracy = training_epoch(
            model, optimizer, criterion, train_loader,
            tqdm_desc=f'Training {epoch}/{num_epochs}'
        )
        test_loss, test_accuracy = validation_epoch(
            model, criterion, test_loader,
            tqdm_desc=f'Validating {epoch}/{num_epochs}'
        )

        if scheduler is not None:
            scheduler.step()

        train_losses += [train_loss]
        train_accuracies += [train_accuracy]
        test_losses += [test_loss]
        test_accuracies += [test_accuracy]

    return train_losses, test_losses, train_accuracies, test_accuracies

In [8]:
#!g1.1
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights

num_epochs = 60
model = mobilenet_v2(num_classes=len(train_dataset.classes)).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_epochs)

In [9]:
#!g1.1
train_losses, test_losses, train_accuracies, test_accuracies = train(
    model, optimizer, scheduler, criterion, train_loader, test_loader, num_epochs
)

Training 1/10:   0%|          | 0/2344 [00:00<?, ?it/s]

Validating 1/10:   0%|          | 0/782 [00:00<?, ?it/s]

Training 2/10:   0%|          | 0/2344 [00:00<?, ?it/s]

Validating 2/10:   0%|          | 0/782 [00:00<?, ?it/s]

Training 3/10:   0%|          | 0/2344 [00:00<?, ?it/s]

...

Validating 8/10:   0%|          | 0/782 [00:00<?, ?it/s]

Training 9/10:   0%|          | 0/2344 [00:00<?, ?it/s]

Validating 9/10:   0%|          | 0/782 [00:00<?, ?it/s]

Training 10/10:   0%|          | 0/2344 [00:00<?, ?it/s]

Validating 10/10:   0%|          | 0/782 [00:00<?, ?it/s]

In [10]:
#!g1.1
test_accuracies

[0.10548,
 0.17552,
 0.25164,
 0.3128,
 0.36552,
 0.40508,
 0.44676,
 0.4714,
 0.48836,
 0.49648]

In [11]:
#!g1.1
final_test = BhwtestDataset('/home/jupyter/mnt/datasets/bhw1/test/test/', transform=test_transform)
final_loader = DataLoader(final_test, batch_size=1, shuffle=False, num_workers=4)

In [12]:
#!g1.1
import gc

@torch.no_grad()
def get_pred(model, images):
    images = images.to(device)
    return model(images)


In [14]:
#!g1.1
ans_labels = []
model.eval()
with torch.no_grad():
    for images, labels in tqdm(final_loader):
        images = images.to(device)
        ans_labels.append(model(images))

  0%|          | 0/10000 [00:00<?, ?it/s]

In [27]:
#!g1.1
pred = []
for lab in ans_labels:
    pred.append(lab.argmax(dim=1).item())


In [33]:
#!g1.1
Label = np.array(pred)
Id = final_test.file_names

In [43]:
#!g1.1
labels_test = pd.DataFrame()
labels_test['Id'] = Id
labels_test['Label'] = Label


#, columns=['Id', 'Label']

In [46]:
#!g1.1
labels_test.to_csv('labels_test.csv', index=False)

In [ ]:
#!g1.1
